## 3.3.2 Hybrid Search Example

### 1. Hybrid Search 1

In [1]:
! pip install llama_index.vector_stores.faiss
! pip install faiss-cpu

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.9 MB 9.2 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/11.9 MB 9.1 MB/s eta 0:00:01
   ------------------- -------------------- 5.8/11.9 MB 9.3 MB/s eta 0:00:01
   -------------------------- ------------- 7.9/11.9 MB 9.4 MB/s eta 0:00:01
   --------------------------------- ------ 10.0/11.9 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------  11.8/11.9 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 11.9/11.9 MB 8.7 MB/s  0:00:01
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 10.9 MB/s  0:00:00
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)

  Attempting uninstall: setuptools

    Found existing installation: setuptools 63.2.0

   --- ---------------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.12.19 requires llama-index-core<0.13.0,>=0.12.19, but you have llama-index-core 0.14.2 which is incompatible.
llama-index-agent-openai 0.4.6 requires llama-index-core<0.13.0,>=0.12.18, but you have llama-index-core 0.14.2 which is incompatible.
llama-index-cli 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.14.2 which is incompatible.
llama-index-embeddings-clip 0.3.1 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.14.2 which is incompatible.
llama-index-embeddings-openai 0.3.1 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.14.2 which is incompatible.
llama-index-indices-managed-llama-cloud 0.6.8 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.14.2 wh

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---- ----------------------------------- 1.8/18.2 MB 10.1 MB/s eta 0:00:02
   ---------- ----------------------------- 4.7/18.2 MB 10.6 MB/s eta 0:00:02
   ------------- -------------------------- 6.0/18.2 MB 9.5 MB/s eta 0:00:02
   ----------------- ---------------------- 8.1/18.2 MB 9.5 MB/s eta 0:00:02
   --------------------- ------------------ 10.0/18.2 MB 9.4 MB/s eta 0:00:01
   ------------------------- -------------- 11.5/18.2 MB 8.8 MB/s eta 0:00:01
   ---------------------------- ----------- 13.1/18.2 MB 8.7 MB/s eta 0:00:01
   -------------------------------- ------- 14.9/18.2 MB 8.6 MB/s eta 0:00:01
   ------------------------------------ --- 16.5/18.2 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------  18.1/18.2 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 8.2 MB/s  0:00:02
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ----

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-clip 0.3.1 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.14.2 which is incompatible.
llama-index-embeddings-openai 0.3.1 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.14.2 which is incompatible.
llama-index-indices-managed-llama-cloud 0.6.8 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.14.2 which is incompatible.
llama-index-llms-openai 0.3.20 requires llama-index-core<0.13.0,>=0.12.17, but you have llama-index-core 0.14.2 which is incompatible.
llama-index-multi-modal-llms-openai 0.4.3 requires llama-index-core<0.13.0,>=0.12.3, but you have llama-index-core 0.14.2 which is incompatible.
llama-index-program-openai 0.3.1 requires llama-index-core<0.13.0,>=0.12.0, but you

In [1]:
### Step1. 문서 준비
sample_documents = [
    "RAG 기술의 최신 동향",
    "최신 검색 증강 시스템 연구",
    "RAG을 활용한 챗봇 개발",
    "RAG와 Dense Passage Retrieval의 비교 연구",
    "최신 AI 논문: 검색 증강 기술",
]

### Document 객체 생성
from llama_index.core import Document
documents = [Document(text=doc) for doc in sample_documents]

In [2]:
### Step2. 희소 검색을 위한 형태소 분석기 함수 작성
from konlpy.tag import Okt
okt = Okt() # Okt 형태소 분석기 초기화
def tokenize_korean_text(text): # 문서 내용 토큰화 함수 정의
    return okt.morphs(text)  # 한국어 형태소 기반 토큰화

In [ ]:

### Step3.  LLM 및 Embedding Model 설정
## API KEY 설정
from dotenv import load_dotenv
load_dotenv()

## 모델 설정
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
Settings.llm = OpenAI(model="gpt-4o", temperature=0.5)  # 모델명은 예시
embedding_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [5]:
### Step4. 청킹을 위한 Splitter 설정
# 청킹 (chunking)
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [6]:
### Step5. Keyword( BM25 ) Index 설정
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.indices.keyword_table import KeywordTableIndex
keyword_index = KeywordTableIndex.from_documents(
    documents=documents,
    text_splitter=splitter,
    extract_keyword=tokenize_korean_text,
    show_progress=True,
)
## bm25 retriever 객체 생성(keyword_index 연결)
bm25_retriever = BM25Retriever.from_defaults(
    index=keyword_index,
    similarity_top_k=2,
)


c:\Users\USER\OneDrive\Desktop\Project\llamaindex_practice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


resource module not available on Windows


Extracting keywords from nodes: 100%|██████████| 5/5 [00:09<00:00,  1.99s/it]


In [7]:
### Step6. 임베딩 모델을 사용 한  VectorStoreIndex 생성
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex.from_documents(
    documents=documents,
    text_splitter=splitter,
    embed_model=embedding_model
)
# dense retriever 객체 생성(vector_index 연결)
semantic_retriever = vector_index.as_retriever(similarity_top_k=2)


In [8]:
### Step7. 인덱스 db 구축
vector_index.storage_context.persist(persist_dir="./index/ch03_hybrid_search_vertor_storage")
keyword_index.storage_context.persist(persist_dir="./index/ch03_hybrid_search_keyword_storage")


In [9]:
### Step8. Hybrid Search 함수 구현
def hybrid_search(query, bm25_retriever, semantic_retriever, 
                bm25_weight=0.5, semantic_weight=0.5):
    # BM25 검색 결과
    bm25_results = bm25_retriever.retrieve(query)
    bm25_scores = {node.text: node.score * bm25_weight for node in bm25_results}
    
    # Semantic 검색 결과
    semantic_results = semantic_retriever.retrieve(query)
    semantic_scores = {node.text: node.score * semantic_weight for node in semantic_results}

    # Hybrid Score 결합
    combined_scores = bm25_scores.copy()
    for node_text, score in semantic_scores.items():
        if node_text in combined_scores:
            combined_scores[node_text] += score
        else:
            combined_scores[node_text] = score

    # 스코어 정렬
    sorted_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_results

In [10]:
### Step8. 쿼리 검색 수행
query = "검색 증강 생성 기술에 대해 알려주세요"
search_results = hybrid_search(query, bm25_retriever, semantic_retriever, bm25_weight=0.5, semantic_weight=0.5)

print("[Hybrid Search Results]")
for rank, (node, score) in enumerate(search_results, start=1):
    print(f"{rank}. {node} (score: {score})")

[Hybrid Search Results]
1. 최신 검색 증강 시스템 연구 (score: 0.8014721855825501)
2. 최신 AI 논문: 검색 증강 기술 (score: 0.7702755156728551)
